# 🔍 Debug: Web API vs Notebook CIF Selection

This notebook helps debug why **"No phases identified"** occurs in the web portal but phases were found yesterday in notebook.

## Problem Statement
- **Web Portal**: No phases identified ❌
- **Yesterday's Notebook**: 3 phases found ✅
- **Database**: ICSD
- **Elements**: Y, Cl (assumed)

## Goal
Compare which CIF files are selected by:
1. Web API (`prepare_phases_for_dara`)
2. Notebook workflow (yesterday's method)


## 1. Import Required Libraries

In [1]:
import sys
from pathlib import Path
import pandas as pd
import requests
import json

# Add scripts to path to use dara_adapter
repo_root = Path.cwd().parent
scripts_path = repo_root / "scripts"
sys.path.insert(0, str(scripts_path))

from dara_adapter import prepare_phases_for_dara

print("✅ Libraries imported successfully")
print(f"📁 Repo root: {repo_root}")
print(f"📁 Scripts path: {scripts_path}")


✅ Libraries imported successfully
📁 Repo root: d:\Haiwen\Code_Repositories\dara
📁 Scripts path: d:\Haiwen\Code_Repositories\dara\scripts


## 2. 配置测试参数

**修改下面的参数来测试你的化学系统：**

注意：
- 现在索引文件已经恢复正常！✅
- 可以直接运行，不需要启动Web服务器
- 这个notebook会直接测试 `prepare_phases_for_dara()` 函数

In [3]:
# 🔧 修改这里：填写你要测试的元素
TEST_DATABASE = "ICSD"  # 可选: "ICSD", "COD", "MP"
TEST_ELEMENTS = ["Y", "Mo", "O"]  # 改成你要测试的元素
MAX_PHASES = 500

print(f"🧪 测试配置:")
print(f"   数据库: {TEST_DATABASE}")
print(f"   元素: {TEST_ELEMENTS}")
print(f"   最大相数: {MAX_PHASES}")

🧪 测试配置:
   数据库: ICSD
   元素: ['Y', 'Mo', 'O']
   最大相数: 500


## 3. 从数据库索引中获取CIF文件

使用 `prepare_phases_for_dara()` 函数筛选CIF文件。

In [4]:
# 选择索引文件
index_files = {
    "ICSD": repo_root / "indexes" / "icsd_index_filled.parquet",
    "COD": repo_root / "indexes" / "cod_index_filled.parquet",
    "MP": repo_root / "indexes" / "mp_index.parquet",
}

index_path = index_files[TEST_DATABASE]

print(f"📖 获取CIF文件列表...")
print(f"   数据库: {TEST_DATABASE}")
print(f"   索引: {index_path.name}")
print(f"   元素: {TEST_ELEMENTS}")

# 调用 prepare_phases_for_dara 获取CIF列表
cif_paths = prepare_phases_for_dara(
    index_path=index_path,
    required_elements=TEST_ELEMENTS,
    exclude_elements=[],
    max_phases=MAX_PHASES,
)

cif_files = [Path(p).name for p in cif_paths]

print(f"\n✅ 结果:")
print(f"   找到 {len(cif_paths)} 个CIF文件")

if len(cif_paths) > 0:
    print(f"\n📋 前20个CIF文件:")
    for i, (name, path) in enumerate(zip(cif_files[:20], cif_paths[:20]), 1):
        exists = "✓" if Path(path).exists() else "✗"
        print(f"   {i:2d}. {name:30s} [{exists}]")
    
    if len(cif_files) > 20:
        print(f"   ... 还有 {len(cif_files) - 20} 个文件")
else:
    print(f"\n⚠️ 没有找到任何CIF文件！")
    print(f"   可能原因：")
    print(f"     - 索引中没有包含所有这些元素的结构")
    print(f"     - 元素名称输入错误")


📖 获取CIF文件列表...
   数据库: ICSD
   索引: icsd_index_filled.parquet
   元素: ['Y', 'Mo', 'O']

✅ 结果:
   找到 194 个CIF文件

📋 前20个CIF文件:
    1. 4111.cif                       [✓]
    2. 15535.cif                      [✓]
    3. 15814.cif                      [✓]
    4. 15815.cif                      [✓]
    5. 15903.cif                      [✓]
    6. 16394.cif                      [✓]
    7. 16574.cif                      [✓]
    8. 18311.cif                      [✓]
    9. 21107.cif                      [✓]
   10. 23722.cif                      [✓]
   11. 23811.cif                      [✓]
   12. 24031.cif                      [✓]
   13. 24032.cif                      [✓]
   14. 24033.cif                      [✓]
   15. 24322.cif                      [✓]
   16. 24559.cif                      [✓]
   17. 24577.cif                      [✓]
   18. 26190.cif                      [✓]
   19. 27202.cif                      [✓]
   20. 27503.cif                      [✓]
   ... 还有 174 个文件

✅ 结果:
   找到 194 个C

## 4. 检查CIF文件元数据

读取前10个CIF文件，验证化学式和元素是否正确。

In [5]:
from pymatgen.core import Structure

if len(cif_paths) == 0:
    print("⚠️ 没有CIF文件可以检查")
else:
    print(f"📖 读取前10个CIF文件的元数据...\n")
    
    cif_metadata = []
    for i, cif_path in enumerate(cif_paths[:10], 1):
        try:
            structure = Structure.from_file(cif_path)
            formula = structure.composition.reduced_formula
            elements = sorted([str(el) for el in structure.composition.elements])
            
            cif_metadata.append({
                'filename': Path(cif_path).name,
                'formula': formula,
                'elements': ', '.join(elements),
                'num_sites': len(structure),
            })
            
            print(f"{i:2d}. {Path(cif_path).name}")
            print(f"    化学式: {formula}")
            print(f"    元素: {', '.join(elements)}")
            print(f"    原子数: {len(structure)}")
            print()
            
        except Exception as e:
            print(f"{i:2d}. {Path(cif_path).name} - ❌ 错误: {e}\n")
    
    # 转换为DataFrame
    if cif_metadata:
        df_metadata = pd.DataFrame(cif_metadata)
        print("\n" + "=" * 80)
        print("📊 元数据汇总表:")
        print("=" * 80)
        print(df_metadata.to_string(index=False))

📖 读取前10个CIF文件的元数据...

 1. 4111.cif
    化学式: Mo17O47
    元素: Mo5+, Mo6+, O2-
    原子数: 128

 2. 15535.cif
    化学式: O2
    元素: O0+
    原子数: 6

 3. 15814.cif
    化学式: Mo4O11
    元素: Mo5.50+, O2-
    原子数: 60

 4. 15815.cif
    化学式: Mo4O11
    元素: Mo5.50+, O2-
    原子数: 60

 5. 15903.cif
    化学式: Mo3O
    元素: Mo0+, O0+
    原子数: 16

 6. 16394.cif
    化学式: Y2O3
    元素: O2-, Y3+
    原子数: 80

 7. 16574.cif - ❌ 错误: Invalid CIF file with no structures!

 8. 18311.cif
    化学式: O2
    元素: O0+
    原子数: 4

 9. 21107.cif
    化学式: O2
    元素: O0+
    原子数: 64

10. 23722.cif
    化学式: MoO2
    元素: Mo4+, O2-
    原子数: 12


📊 元数据汇总表:
 filename formula        elements  num_sites
 4111.cif Mo17O47 Mo5+, Mo6+, O2-        128
15535.cif      O2             O0+          6
15814.cif  Mo4O11    Mo5.50+, O2-         60
15815.cif  Mo4O11    Mo5.50+, O2-         60
15903.cif    Mo3O       Mo0+, O0+         16
16394.cif    Y2O3        O2-, Y3+         80
18311.cif      O2             O0+          4
21107.cif      O2       

d:\Haiwen\Code_Repositories\dara\.venv\Lib\site-packages\pymatgen\io\cif.py:1313: UserWarning: Some occupancies ([2.0, 2.0]) sum to > 1! If they are within the occupancy_tolerance, they will be rescaled. The current occupancy_tolerance is set to: 1.0
  if struct := self._get_structure(data, primitive, symmetrized, check_occu=check_occu):
d:\Haiwen\Code_Repositories\dara\.venv\Lib\site-packages\pymatgen\core\structure.py:3102: UserWarning: No structure parsed for section 1 in CIF.
Occupancy 2.0 exceeded tolerance.
  struct = parser.parse_structures(primitive=primitive)[0]
d:\Haiwen\Code_Repositories\dara\.venv\Lib\site-packages\pymatgen\core\structure.py:3102: UserWarning: Issues encountered while parsing CIF: Some occupancies ([2.0, 2.0]) sum to > 1! If they are within the occupancy_tolerance, they will be rescaled. The current occupancy_tolerance is set to: 1.0
No structure parsed for section 1 in CIF.
Occupancy 2.0 exceeded tolerance.
  struct = parser.parse_structures(primitive=prim

## 5. 查看索引数据库内容

直接读取索引文件，查看原始数据。

In [ ]:
# 读取索引文件
df_index = pd.read_parquet(index_path)

print(f"? {TEST_DATABASE} 索引统计:")
print(f"   总记录数: {len(df_index):,}")
print(f"   列: {list(df_index.columns)}")

# 筛选包含所有测试元素的记录
if 'elements' in df_index.columns:
    print(f"\n🔍 筛选包含 {TEST_ELEMENTS} 的记录...")
    
    def contains_all_elements(elements_value, required):
        """检查是否包含所有必需元素"""
        if pd.isna(elements_value):
            return False
        # 处理字符串或列表格式
        if isinstance(elements_value, str):
            elements_set = set(elements_value.split(','))
        elif isinstance(elements_value, list):
            elements_set = set(elements_value)
        else:
            return False
        return all(el in elements_set for el in required)
    
    filtered = df_index[df_index['elements'].apply(lambda x: contains_all_elements(x, TEST_ELEMENTS))]
    
    print(f"✅ 找到 {len(filtered)} 条记录")
    
    if len(filtered) > 0:
        print(f"\n📋 前10条匹配记录:")
        # 选择要显示的列
        if 'id' in filtered.columns:
            display_cols = ['id', 'formula', 'elements', 'spacegroup']
        else:
            display_cols = ['formula', 'elements', 'spacegroup']
        
        # 确保列存在
        display_cols = [c for c in display_cols if c in filtered.columns]
        
        print(filtered[display_cols].head(10).to_string(index=False))
    else:
        print(f"\n⚠️ 索引中没有同时包含 {TEST_ELEMENTS} 的记录")
else:
    print("\n⚠️ 索引中没有'elements'列")
    print(f"可用列: {list(df_index.columns)}")

: 

## 6. 总结与建议

基于以上测试结果的分析。

In [ ]:
print("=" * 80)
print("? 测试总结")
print("=" * 80)

print(f"\n测试配置:")
print(f"  数据库: {TEST_DATABASE}")
print(f"  元素: {TEST_ELEMENTS}")
print(f"  找到CIF: {len(cif_paths)} 个")

print(f"\n✅ 检查项目:")

# 1. CIF数量检查
if len(cif_paths) == 0:
    print(f"  ❌ 1. 没有找到任何CIF文件")
    print(f"       可能原因:")
    print(f"       - 索引中没有同时包含所有元素的结构")
    print(f"       - 元素名称拼写错误")
    print(f"       建议: 尝试减少元素数量，或者换其他数据库")
else:
    print(f"  ✅ 1. 成功找到 {len(cif_paths)} 个CIF文件")

# 2. 文件存在性检查
if len(cif_paths) > 0:
    missing = [p for p in cif_paths if not Path(p).exists()]
    if missing:
        print(f"  ⚠️ 2. 有 {len(missing)} 个CIF文件不存在")
        print(f"       缺失文件示例: {missing[0]}")
    else:
        print(f"  ✅ 2. 所有CIF文件都存在")

# 3. 路径格式检查
if len(cif_paths) > 0:
    absolute_count = sum(1 for p in cif_paths if Path(p).is_absolute())
    if absolute_count == len(cif_paths):
        print(f"  ✅ 3. 所有路径都是绝对路径")
    else:
        print(f"  ⚠️ 3. 有相对路径存在 ({len(cif_paths) - absolute_count} 个)")

print(f"\n💡 下一步建议:")

if len(cif_paths) > 0:
    print(f"  ✅ CIF文件列表正常，可以用于相分析")
    print(f"  建议:")
    print(f"     1. 在 streamlined_phase_analysis.ipynb 中使用这些元素")
    print(f"     2. 或者在网页端输入相同的元素: {', '.join(TEST_ELEMENTS)}")
    print(f"     3. 确保XRD图谱文件正确")
else:
    print(f"  ⚠️ 需要调整搜索条件")
    print(f"  建议:")
    print(f"     1. 尝试单个元素或双元素系统")
    print(f"     2. 换其他数据库 (COD/MP/ICSD)")
    print(f"     3. 检查元素名称拼写")

print("\n" + "=" * 80)